## Importação das bibliotecas

In [36]:
import numpy as np
import pandas as pd
import plotly
import plotly.offline as py
import yfinance as yf
import datetime
import plotly.graph_objs as go
import openpyxl as xl
import yagmail

## Sem esse comando os graficos não sao exibidos no notebook

In [37]:
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode(connected=True)

## Recebe os inputs do usuário e busca as cotações na base de dados do Yahoo

In [38]:
# Recebe o input do usuario com o nome da ação
stock = input('Digite o código da ação que deseja analisar')
stock = stock + '.SA'
# Recebe do usuário o valor das médias 
num_mm_short = int(input('Digite um valor númerico para a média móvel curta'))
num_mm_long = int(input('Digite um valor numerico para a média móvel longa'))
mean_short = 'MM' + str(num_mm_short)
mean_long = 'MM' + str(num_mm_long)
# Define o lote padrão de 100 ações
capital = int(input('Digite o capital disponível para essa estratégia'))

# Define o periodo de busca do historico de preços
ini_date = input('Digite a data inicial da busca.(Formato:DD-MM-AAAA')
end_date = input('Digite a data inicial da busca.(Formato:DD-MM-AAAA')
ini_date = ini_date.replace('/','-')
end_date = end_date.replace('/','-')
ini_date = datetime.datetime.strptime(ini_date, "%d-%m-%Y")
end_date = datetime.datetime.strptime(end_date, "%d-%m-%Y")

# Digitação do email e senha caso queira receber alerta de entrada por email
email = input('Digite seu email')
password = input('Digite a senha do seu email. Caso não queria receber alerta via e-mail digitar 0 para a senha')
yag = yagmail.SMTP(email,password)

df_stock_prices = yf.download(stock, start=ini_date, end=end_date, interval='1d')
df_stock_prices.reset_index(drop=False, inplace=True)

df_stock_prices.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,36.950001,37.320000,36.820000,37.320000,31.196165,13303600.0
1,2010-01-05,37.380001,37.430000,36.799999,37.000000,30.928682,21396400.0
2,2010-01-06,36.799999,37.500000,36.799999,37.500000,31.346632,18720600.0
3,2010-01-07,37.270000,37.450001,37.070000,37.150002,31.054062,10964600.0
4,2010-01-08,37.160000,37.389999,36.860001,36.950001,30.886881,14624200.0


## Cria as colunas de média móvel no Data Frame e preenche com zero os campos com 'NaN'

In [39]:
df_stock_prices['MM_SHORT'] = df_stock_prices.Close.rolling(window=num_mm_short).mean()
df_stock_prices['MM_LONG'] = df_stock_prices.Close.rolling(window=num_mm_long).mean()
df_stock_prices['SPREAD_MM'] = df_stock_prices['MM_SHORT'] - df_stock_prices['MM_LONG']

df_stock_prices['Start'] = 0.00
df_stock_prices['Stop'] = 0.00
df_stock_prices['Profit'] = 0.00

df_stock_prices['MM_SHORT'] = df_stock_prices['MM_SHORT'].fillna(0)
df_stock_prices['MM_LONG'] = df_stock_prices['MM_LONG'].fillna(0)
# Excluindo do Data frame os registros com a média móvel zerada pois não dá pra fazer backtest nesses registros
df_stock_prices = df_stock_prices.loc[df_stock_prices['MM_LONG']!=0]
df_stock_prices.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,MM_SHORT,MM_LONG,SPREAD_MM,Start,Stop,Profit
49,2010-03-17,37.430000,37.480000,36.930000,37.150002,31.054062,10298100.0,35.510000,35.1366,0.373400,0.0,0.0,0.0
50,2010-03-18,37.139999,37.400002,36.759998,37.020000,30.945391,12061700.0,35.641905,35.1306,0.511305,0.0,0.0,0.0
51,2010-03-19,37.000000,37.090000,36.160000,36.230000,30.285025,24306500.0,35.715238,35.1152,0.600038,0.0,0.0,0.0
52,2010-03-22,35.610001,36.360001,35.500000,36.240002,30.293386,13489200.0,35.805238,35.0900,0.715238,0.0,0.0,0.0
53,2010-03-23,36.380001,36.419998,35.709999,35.820000,29.942307,17207000.0,35.860953,35.0634,0.797552,0.0,0.0,0.0
54,2010-03-24,35.799999,36.570000,35.599998,36.090000,30.167995,18683500.0,35.951429,35.0462,0.905229,0.0,0.0,0.0
55,2010-03-25,36.330002,36.430000,35.200001,35.200001,29.424036,17698800.0,36.011429,35.0136,0.997829,0.0,0.0,0.0
56,2010-03-26,35.410000,35.549999,34.299999,34.500000,28.838902,28820300.0,36.017143,34.9764,1.040743,0.0,0.0,0.0
57,2010-03-29,34.740002,35.200001,34.119999,34.900002,29.173264,19312800.0,36.030953,34.9484,1.082553,0.0,0.0,0.0
58,2010-03-30,35.090000,35.299999,34.759998,34.799999,29.089678,12365900.0,36.024762,34.9310,1.093762,0.0,0.0,0.0


## Plota o gráfico com o preço de fechamento e as médias móveis

In [40]:
close_prices = go.Scatter(
                            x = df_stock_prices['Date'],
                            y = df_stock_prices['Close'],
                            name = stock + 'Preço de Fechamento',
                            line = dict(color = '#330000'),
                            opacity = 1.0)

close_prices2 = go.Candlestick(
                                x = df_stock_prices['Date'],
                                open = df_stock_prices['Open'],
                                high = df_stock_prices['High'],
                                low = df_stock_prices['Low'],
                                close = df_stock_prices['Close'])

mm_s = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM_SHORT'],
                    name = 'Média Móvel Curta',
                    line = dict(color = '#B2FF66'),
                    opacity = 1.0)

mm_l = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM_LONG'],
                    name = 'Média Móvel Longa',
                    line = dict(color = '#FF00FF'),
                    opacity = 1.0)

layout = go.Layout(
                    autosize=False,
                    legend_orientation='h',
                    width=1000,
                    height=400)

data_prices = [close_prices,mm_s,mm_l]

fig = go.Figure(data=data_prices, layout=layout)

py.iplot(fig)

## Faz o backtest da estratégia 

In [41]:
start_price = 0
last_spread = 0
lowest_price = 0
drawdown = 0
drawdown2 = 0
payoff = 0
date_today = datetime.date.today() - datetime.timedelta(days=3)
send_mail = False
fee = 5
fee_value = 0
income_tax = 0
capital = 5000


for row in df_stock_prices.itertuples():

    lote = capital / row.Close
    verify_today = row.Date >= date_today
    # Se a média curta cruzou a média longa e não há nenhuma operação em aberto é feita uma operação
    if (row.MM_SHORT > row.MM_LONG) & (start_price < 0.01):
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
        if verify_today == True:
            yag.send(email, 'Estratégia Cruzamento MM - Algo Python', 'Entrada pela estratégia de cruzmento de MM foi detectada')
        send_mail = False
        lowest_price = row.Close
    # Se a média curta está abaixo da média longa e há uma operação em aberto não fazer nada
    elif (row.MM_SHORT < row.MM_LONG) & (start_price < 0.01):
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
        if row.Close < lowest_price:
            lowest_price = row.Close
    # Se a média móvel curta está acima da mm longa e existe uma operação em aberto é verificado se houve cruzamento
    # da mm curta pra baixo da longa 
    elif (row.MM_SHORT > row.MM_LONG) & (start_price > 0):
        # Se o spread é menor que 0 é porque houve cruzamento pra baixo da mm curta com a mm longa
        if last_spread < 0:
            df_stock_prices.at[row.Index, 'Profit'] = (start_price - row.Close) * lote
            capital = capital + ((row.Close - start_price) * lote)
            df_stock_prices.at[row.Index, 'Stop'] = row.Close
            if row.Close < lowest_price:
                lowest_price = row.Close
            # Calcula o drawdown
            if row.Profit < 0 :
                drawdown2 = (start_price - lowest_price) / start_price
                if drawdown2 > drawdown:
                    drawdown = drawdown2
                    drawdown2 = 0
            df_stock_prices.at[row.Index, 'Start'] = row.Close
            start_price = row.Start
        # Se o spread NÃO é menor ele não faz nada
        else:
            df_stock_prices.at[row.Index, 'Start'] = start_price
    elif (row.MM_SHORT < row.MM_LONG) & (start_price > 0):
        if last_spread > 0:
            df_stock_prices.at[row.Index, 'Profit'] = (row.Close - start_price) * lote
            capital = capital + ((row.Close - start_price) * lote)
            df_stock_prices.at[row.Index, 'Stop'] = row.Close
            if row.Close < lowest_price:
                lowest_price = row.Close
            #Calcula o drawdown
            if row.Profit < 0 :
                drawdown2 = (start_price - lowest_price) / start_price
                if drawdown2 > drawdown:
                    drawdown = drawdown2
                    drawdown2 = 0
            df_stock_prices.at[row.Index, 'Start'] = row.Close
            start_price = row.Start
            if verify_today == True:
                yag.send(email, 'Estratégia Cruzamento MM - Algo Python', 'Encerrada operação pela estratégia de                                      cruzamento de MM.')
            #send_mail = False
        else:
            df_stock_prices.at[row.Index, 'Start'] = start_price
            if verify_today == True:
                yag.send(email, 'Estratégia Cruzamento MM - Algo Python', 'Entrada pela estratégia de cruzamento de                                   MM foi detectada')
            #send_mail = False
    last_spread = row.SPREAD_MM


#df_stock_prices.head(10)

## Print mostrando o resultado da estrategia

In [42]:
print('O resultado da estratégia é R$ %7.2f' % df_stock_prices['Profit'].sum())

O resultado da estratégia é R$ 3356.84


## Exporta o data frame para uma planilha do excel

In [43]:
df_stock_prices.to_excel('cruzamento_mm.xlsx', sheet_name='Cross_MM', index=False)

In [44]:
# Armazena numero de entradas da estratégia
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']!=0].count()
entry_count = df_stock_prices_count['Profit']

# Armazena número de entradas com lucro
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']>0].count()
entry_count_profit = df_stock_prices_count['Profit']

# Armazena número de entradas com prejuizo
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']<0].count()
entry_count_loss = df_stock_prices_count['Profit']

# Armazena resultado financeiro das entradas com lucro
df_stock_prices_sum = df_stock_prices.loc[df_stock_prices['Profit']>0].sum()
entry_sum_profit = df_stock_prices_sum['Profit']

# Armazena resultado financeiro das entradas com prejuizo
df_stock_prices_sum = df_stock_prices.loc[df_stock_prices['Profit']<0].sum()
entry_sum_loss = df_stock_prices_sum['Profit']

# Payoff
payoff = (entry_sum_profit / entry_count_profit) / ((entry_sum_loss / entry_count_loss) * -1)
# Taxa de acerto
hit_rate = entry_count_profit / entry_count
# Expectativa matemática
math_expec = payoff * (1 - hit_rate)
# Corretagem
fee_value = (fee * entry_count) * 2
# Se houver lucro calcula o valor do IR
if df_stock_prices['Profit'].sum() > 0:
    income_tax = df_stock_prices['Profit'].sum() * 0.2

print(hit_rate)
print(payoff)
print(math_expec)
print('O número de entradas executadas foi de %7.0f' % entry_count)


0.36363636363636365
5.578380485460412
3.549878490747535
O número de entradas executadas foi de      44


In [45]:
# Se o arquivo existe ele será excluido para ser criado novamente
if os.path.exists('cruzamento_mm.xlsx'):
  os.remove('cruzamento_mm.xlsx')
else:
  print("The file does not exist")

# Exporta o data frame para um arquivo no Excel
df_stock_prices.to_excel('cruzamento_mm.xlsx', sheet_name='cruzamento_mm', index=False)

# Cria um resumo do backtest em uma nova guia do Excel
wb = xl.load_workbook('cruzamento_mm.xlsx')

sheet = wb.create_sheet(index = 0 , title = "Resumo_Backtest") 

sheet.column_dimensions['A'].width = 30
sheet.column_dimensions['B'].width = 20
sheet['A1'] = 'Resumo do Backtest'
sheet['A2'] = 'Periodo avaliado:'
sheet['B2'] = str(ini_date) + ' até ' + str(end_date)
sheet['A3'] = 'Papel:'
sheet['B3'] = stock.upper()
sheet['A4'] = 'Capital:'
sheet['B4'] = capital
sheet['B4'].style = 'Comma'
sheet['A5'] = 'Média Móvel Curta:'
sheet['B5'] = num_mm_short
sheet['A6'] = 'Média Móvel Longa'
sheet['B6'] = num_mm_long
sheet['A7'] = 'Resultado Financeiro: R$'
sheet['B7'] = df_stock_prices['Profit'].sum()
sheet['B7'].style = 'Comma'
sheet['A8'] = 'Rentabilidade:'
sheet['B8'] = df_stock_prices['Profit'].sum() / capital
sheet['B8'].style = 'Percent'
sheet['A9'] = 'Número de entradas Executadas: '
sheet['B9'] = entry_count
sheet['A10'] = 'Número de operações Vencedoras:'
sheet['B10'] = entry_count_profit
sheet['A11'] = 'Número de operações Perdedoras:'
sheet['B11'] = entry_count_loss
sheet['A12'] = 'O payoff da estratégia é:'
sheet['B12'] = payoff
sheet['B12'].style = 'Comma'
sheet['A13'] = 'O drawdown máximo da estratégia é de:'
sheet['B13'] = drawdown
sheet['B13'].style = 'Percent'
sheet['A14'] = 'A taxa de acerto da estratégia é de:'
sheet['B14'] = hit_rate
sheet['B14'].style = 'Percent'
sheet['A15'] = 'A expectativa matemática da estratégia é de:'
sheet['B15'] = math_expec
sheet['B15'].style = 'Comma'
sheet['A16'] = 'Valor gasto com corretagem:'
sheet['B16'] = fee_value
sheet['B16'].style = 'Comma'
sheet['A17'] = 'O valor de IR é:'
sheet['B17'] = income_tax
sheet['B17'].style = 'Comma'
sheet['A18'] = 'Resultado líquido após corretagem e IR:'
sheet['B18'] = capital + df_stock_prices['Profit'].sum() - fee_value - income_tax
sheet['B18'].style = 'Comma'
wb.save('cruzamento_mm.xlsx')